In [1]:
# Only need to do this once...
!pip install flask
!pip install --upgrade google-cloud-datastore
!pip install Flask-Mail

Requirement already up-to-date: google-cloud-datastore in /usr/local/envs/py3env/lib/python3.5/site-packages (1.15.3)


In [2]:

import json
import re
from flask import Flask, request, jsonify, make_response
from flask import render_template
from google.cloud import datastore

In [ ]:
from send_invitation import send_email 
from threading import Thread
import jwt

app = Flask(__name__)
results = ""
rsize = 0

import re
from random import seed
from random import randint
from datetime import datetime

seed(datetime.now())

app.config['SECRET_KEY'] = 'SECRET_KEY_CHAT9900'

#chatroom public url
def get_chatroom_url():
    # populate this from your local server
    return "http://75789421033d.ngrok.io"

# url of the webhook. 
# it is done by curl the local ngrok api
# It is for friend request validations [out of scope]
def get_public_url():
    tunnel = !curl http://localhost:4040/api/tunnels
    tunnel= str(tunnel)
    print(tunnel)
    mobj = re.search('public_url', tunnel)
    i = mobj.span()
    t = tunnel[i[1]:].split('\"')
    public_url =t[2]
    return public_url

def gen_jwt(seedstr):
    return jwt.encode(
            {'requester': seedstr },
            app.config['SECRET_KEY'], algorithm='HS256')

# [out of scope]
@app.route('/friend/<token>', methods=['GET', 'POST'])
def validate_friend_requests(token):
    meeting = jwt.decode(token, app.config['SECRET_KEY'],
                            algorithms=['HS256'])['requester']
    print("meeting click was clicked ",meeting)
    return render_template('place_holder.html')

@app.route('/webhook/', methods=['POST'])
def handle():
    req = request.get_json(silent=True, force=True)
    print('Request:')
    print(json.dumps(req, indent=4))
    intent = req.get('queryResult').get('intent').get('displayName')
    print(intent)
    # there could be multiple topics in a sentence
    if isinstance(intent, list):
        intent = intent[0]
    #topic = re.sub(r'[^\w\s]', '', topic)
    
    rsp = buildReply('Command not recognized')
    if (intent == "Schedule: Movie - yes"):
        rsp = scheduleMovie(req)
    elif (intent == "profile" or intent == "account"):
        rsp = getProfileAccount()
    elif (intent == "movie recommendation" ):
        rsp = getRecommendations()
    elif (intent == "login-prompt"):
        rsp = validateLogin(req)
        
    rsp = json.dumps(rsp, indent=4)
    print (rsp)
    r = make_response(rsp)
    r.headers['Content-Type'] = 'application/json'
    return r    
  
def get_attendees(current_user_id, current_user_email, tofriend):
    attendees = list()
    attendees.append("chatbot9900@gmail.com")
    attendees.append(current_user_email)
    client = datastore.Client()
    query = client.query(kind="friends")
    query.add_filter("id", "=", current_user_id)
    result = list(query.fetch(1))
    print(current_user_id, result)
    print('fids',result[0].get('fids'))
    friends = result[0].get('fids')
    for f in friends:
        query = client.query(kind="users")
        query.add_filter("id", "=", int(f))
        query.add_filter("name", "=", tofriend)
        print(query)
        print(f, tofriend)
        result = list(query.fetch(1))
        if len(result) == 1 :
            print('email',result[0].get('email'))
            attendees.append(result[0].get('email'))
    return attendees
        
def scheduleMovie(req):
    tofriend = req.get('queryResult').get('outputContexts')[0].get('parameters').get('friend')[0].get('name')
    print(tofriend,'tofriend')
    movie = req.get('queryResult').get('outputContexts')[0].get('parameters').get('movie')[0]
    date = req.get('queryResult').get('outputContexts')[0].get('parameters').get('date')[0]
    time = req.get('queryResult').get('outputContexts')[0].get('parameters').get('time')[0]
    current_user_email = req.get('queryResult').get('outputContexts')[0].get('parameters').get('loggedin_user')
    print(current_user_email, tofriend)
    if (current_user_email == None):
        return {
          "fulfillmentText": 
                  "Please log in first. (Type 'login')"
        }
    
    client = datastore.Client()
    query = client.query(kind="users")
    query.add_filter("email", "=", current_user_email)
    result = list(query.fetch(1))
    if len(result) == 0:
        return {
          "fulfillmentText": 
                  "logged in user email not found. Please contact admin chatbotchatbot9900@gmail.com"
        }
    #print(result)
    #print(result[0])
    #print(result[0].id)
    #print(result[0].get('email'))
    #print(attendees)
    attendees = get_attendees(result[0].get("id"), current_user_email, tofriend)
    #print(str(datetime.now()))
    print(attendees)
    
    # token in this context is just a random string acting as chatroom id
    token = gen_jwt(str(datetime.now()))
    chatroom_url = get_chatroom_url()+"/"+token.decode("utf-8")
    Thread(target=send_email, args=(attendees,chatroom_url, movie, date.split('T')[0], time.split('T')[1]),).start()
    return buildReply('Awesome! Your movie meeting is confirmed. Sending request...')
  
def getProfileAccount():
    urls = ""
    with open('netflix_urls.txt') as netflix_urls:
        urls = netflix_urls.read()
    print (urls)
    return buildReply(urls)

def getRecommendations():
    global results 
    global rsize
    print ("recom")
    if (rsize ==0):
        client = datastore.Client()
        query = client.query(kind='netflix')
        results = list(query.fetch(limit=100))
        rsize = len(results)
        print(rsize)
    t1 = results[randint(0, rsize -1 )]['title'].decode("utf-8") 
    print("t1",t1)
    t2 = results[randint(0, rsize-1)]['title'].decode("utf-8") 
    return buildReply(t1+"\n"+t2)

def validateLogin(req):
    email = req.get('queryResult').get('parameters').get("email")
    password = req.get('queryResult').get('parameters').get("password")
    session = req.get('queryResult').get('outputContexts')[0].get('name')
    client = datastore.Client()
    # you may have to create this kind manually in gcp datastore 
    query = client.query(kind="users")
    print(email,password)
    # These two columns has to be indexed in gcp datastore 
    # otherwise they can not be found and filtered
    query.add_filter("email", "=", email)
    query.add_filter("randomnumber", "=", password)
    result = list(query.fetch(1))
    print("validateLogin",result)
    if len(result) == 1:
        return confirmLogin(email, session)
    return buildReply("Invalid combination")
  
def confirmLogin(un, sn):
    array = sn.split('/')
    session = ""
    for i in range(len(array)-1):
        session += array[i]+'/'
    session += "logged-in"
    return {
      "fulfillmentText": 
              "User: "+un+" logged in"
      ,
      "outputContexts": [
        {
          "name": session,
          "lifespanCount": 500,
          "parameters": {
            "loggedin_user": un
          }
        }
      ]
    }

def buildReply(info):
    return {
        'fulfillmentText': info,
    }

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2021 08:03:20] "POST /webhook/ HTTP/1.1" 200 -


Request:
{
    "responseId": "e32cff7f-fa0f-451b-8315-39e130104cf9-4b8539db",
    "originalDetectIntentRequest": {
        "payload": {}
    },
    "queryResult": {
        "action": "ScheduleMovie.ScheduleMovie-yes",
        "queryText": "Yes, Confirm",
        "intent": {
            "name": "projects/i-backbone-307006/agent/intents/92c4d6e9-70d0-4683-9755-cc9ef3a3fa07",
            "displayName": "Schedule: Movie - yes"
        },
        "sentimentAnalysisResult": {
            "queryTextSentiment": {
                "magnitude": 0.3,
                "score": 0.3
            }
        },
        "allRequiredParamsPresent": true,
        "intentDetectionConfidence": 1.0,
        "languageCode": "en",
        "parameters": {},
        "outputContexts": [
            {
                "name": "projects/i-backbone-307006/agent/sessions/dfMessenger-3074986/contexts/logged-in",
                "parameters": {
                    "time.original": [
                        "3 PM"
         